<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_08_Describe_Related_Images_from_Web_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to search the Web and ingest web pages, crawl image links, and describe the images.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [20]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [ ]:
from typing import List, Optional

# Create specification for Groq Llava 1.5
async def create_groq_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Groq Llava 1.5",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.GROQ,
        groq=input_types.GroqModelPropertiesInput(
            model=enums.GroqModels.LLAVA_1_5_7B,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create specification for OpenAI GPT-4o
async def create_openai_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="OpenAI GPT-4o",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=enums.OpenAIModels.GPT4O_128K,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create specification for Anthropic Sonnet 3.5
async def create_anthropic_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Claude Sonnet 3.5",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create enrichment workflow to crawl only image links from web pages, and enrich them with Groq Llava 1.5
async def create_workflow(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Link Crawling",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.MODEL_IMAGE,
                        modelImage=input_types.ModelImageExtractionPropertiesInput(
                            specification=input_types.EntityReferenceInput(id=specification_id)
                        )
                    )
                )
            ]
        ),
        enrichment=input_types.EnrichmentWorkflowStageInput(
            link=input_types.LinkStrategyInput(
                enableCrawling=True,
                allowContentDomain=True,
                allowedLinks=[enums.LinkTypes.FILE],
                allowedFiles=[enums.FileTypes.IMAGE],
            )
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_feed(search_text: str, read_limit: int, workflow_id: str):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name="Web Search",
        type=enums.FeedTypes.SEARCH,
        search=input_types.SearchFeedPropertiesInput(
            text=search_text,
            readLimit=read_limit
        ),
        workflow=input_types.EntityReferenceInput(
            id=workflow_id
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

# Locate images ingested by feed
async def query_contents(feed_id: str, search: Optional[str] = None):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                search=search,
                searchType=enums.SearchTypes.HYBRID,
                fileTypes=[enums.FileTypes.IMAGE],
                feeds=[
                    input_types.EntityReferenceFilter(
                        id=feed_id
                    )
                ]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)


async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [ ]:
from IPython.display import display, Markdown, Image
import time

# Remove any existing feeds and workflows; only needed for notebook example
await delete_all_workflows()
await delete_all_feeds()

print('Deleted all feeds.')

read_limit = 1 # how many search results to ingest from feed

search_text = "Seattle Kraken mascot Buoy"

# NOTE: uncomment just one of these to use different models
specification_id = await create_groq_specification()
#specification_id = await create_anthropic_specification()
#specification_id = await create_openai_specification()

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        feed_id = await create_feed(search_text, read_limit, workflow_id)

        if feed_id is not None:
            print(f'Created feed [{feed_id}].')

            # Wait for feed to complete, since ingestion happens asychronously
            done = False
            time.sleep(5)
            while not done:
                done = await is_feed_done(feed_id)

                if not done:
                    time.sleep(2)

            print(f'Completed feed [{feed_id}].')

            # Show all the images crawled from the web page
            contents = await query_contents(feed_id)

            if contents is not None:
                print(f'Found {len(contents)} images in feed [{feed_id}].')
                print()

                for content in contents:
                    if content is not None and content.image is not None:
                        display(Image(url=content.image_uri, width=256))

            print()

            # Show just the images related to the search text, and the image text and descriptions
            contents = await query_contents(feed_id, search_text)

            if contents is not None:
                print(f'Found {len(contents)} images in feed [{feed_id}] related to [{search_text}].')
                print()

                for content in contents:
                    if content is not None:
                        display(Markdown(f'## {content.name} [{content.id}]:'))
                        display(Markdown(f'Original image size: {content.image.width}x{content.image.height}'))

                        if content.markdown is not None:
                            display(Markdown(f'Text in image: {content.markdown}'))

                        if content.image is not None:
                            display(Image(url=content.image_uri, width=512))

                            if content.image.description is not None:
                                display(Markdown(f'### Description:\n{content.image.description}'))

Deleted all feeds.
Created specification [b17c88ad-3afa-4f5f-8440-261d2c013e1d].
Created workflow [a648aecb-bfe2-4c76-bc42-2d3f42514d32].
Created feed [3073bda3-6086-4399-bfd0-3ac17afcd797].
Completed feed [3073bda3-6086-4399-bfd0-3ac17afcd797].
Found 15 images in feed [3073bda3-6086-4399-bfd0-3ac17afcd797].




Found 4 images in feed [3073bda3-6086-4399-bfd0-3ac17afcd797] related to [Seattle Kraken mascot Buoy].



## ifttga66jgjxwb82fc58.png [adbd6d91-4bb6-4475-9756-308a2c4107bc]:

Original image size: 720x405

Text in image: BUOY

0



...

### Description:
The image presents a logo design that combines elements of a popular social media platform's branding with custom text. The logo is horizontally oriented, featuring two main components: a stylized camera icon and text.

On the left side of the logo, we see a simplified representation of a camera. This icon is rendered in a gradient color scheme, transitioning from a vibrant pink or magenta at the top left to a warm orange or red at the bottom right. The camera design is minimalist, consisting of a rounded square shape with a smaller circle inside, likely representing the camera lens. This gradient effect gives the icon a modern and dynamic appearance.

To the right of the camera icon, we find text that reads 'NHLBUOY' in a bold, sans-serif typeface. The text is presented in a dark blue or navy color, providing a strong contrast to the colorful camera icon. The font choice appears to be clean and professional, enhancing the logo's readability and impact.

The juxtaposition of the colorful, gradient-filled camera icon with the solid-colored text creates an interesting visual balance. This design choice effectively draws the viewer's attention to both elements of the logo, ensuring that neither overpowers the other.

The use of the camera icon strongly suggests a connection to visual media, photography, or image-sharing platforms. The gradient colors used in the icon are reminiscent of the branding often associated with popular social media applications, particularly those focused on photo and video sharing.

The text 'NHLBUOY' appears to be a unique identifier or brand name. The inclusion of 'NHL' could potentially reference the National Hockey League, though without additional context, this is speculative. 'BUOY' might relate to maritime themes or could be a creative spelling or acronym with significance to the brand.

Overall, this logo design effectively combines recognizable social media visual elements with custom branding. It suggests a platform or service that may be related to visual content sharing, potentially with a specific focus or niche audience as indicated by the unique text component.

The logo's clean design and use of contrasting elements make it visually appealing and memorable. It would likely be effective across various applications, from digital platforms to physical branding materials, due to its simplicity and bold color choices.

The design demonstrates a clever adaptation of familiar social media iconography, potentially aiming to leverage user familiarity with such platforms while establishing its own distinct identity through the custom text and color scheme.

## zdvth9z8rgrd95atnj0j.png [eba3e8db-102d-4a0d-af6a-0e78255553ad]:

Original image size: 720x120

Text in image: SEABUOY



### Description:
The image presents a sleek and modern logo design for a brand called 'SEABUOY'. The logo is composed of two main elements: a geometric symbol and the brand name text.

The symbol on the left side of the logo is a minimalist representation that resembles the letter 'X'. It's constructed from two intersecting lines that form four triangular shapes. This geometric design gives the logo a contemporary and abstract feel, potentially symbolizing precision, strength, or interconnectedness.

To the right of the symbol, the brand name 'SEABUOY' is displayed in bold, uppercase letters. The font choice is sans-serif, which contributes to the modern aesthetic of the overall design. The text is aligned with the symbol, creating a balanced and cohesive look.

The color scheme of the logo is simple yet effective. The 'X' symbol appears in white or light gray, standing out against a dark background. The text 'SEABUOY' is rendered in a deep navy blue color, which immediately evokes associations with the sea, aligning well with the brand name.

The contrast between the light symbol and the dark text creates visual interest and ensures that both elements of the logo are distinct and easily readable. This color choice also suggests a sense of professionalism and reliability.

The name 'SEABUOY' itself is intriguing, as it combines two nautical terms. 'Sea' obviously refers to the ocean, while 'buoy' is a floating device used for navigation or marking specific locations in water. This name suggests that the brand or company is likely related to maritime activities, navigation, or marine technology.

The simplicity of the logo design makes it versatile and scalable, suitable for various applications from small digital icons to large signage. Its clean lines and straightforward presentation ensure that it would remain recognizable even at smaller sizes.

Overall, this logo effectively communicates a brand identity that is modern, professional, and linked to maritime themes. The combination of the abstract symbol and the clear typography creates a memorable visual mark that could represent a company involved in marine technology, navigation systems, or other sea-related industries.